In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import pandas as pd
import time

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.apartments.com/")

In [3]:
home_search_bar = driver.find_element_by_xpath("/html/body/div[1]/div/section[1]/div[1]/section/div/div/input")
home_search_bar.send_keys("Seattle")
home_search_bar.send_keys(Keys.RETURN)
home_search_button = driver.find_element_by_xpath('//*[@id="quickSearch"]/div/div/a')
home_search_button.click()
time.sleep(1)

In [4]:
listing_info = []
while True:
    apartmentslist = driver.find_element_by_xpath('//*[@id="placardContainer"]/ul')
    listing_links = apartmentslist.find_elements_by_tag_name("article")
    for article in listing_links:
        a_tag = article.find_element_by_tag_name("a")
        listing_info.append(a_tag.get_attribute("href"))
    try:
        nextbtn = driver.find_element_by_css_selector("a.next")
        nextbtn.click()
        time.sleep(1)
    except:
        print("No more pages remaining exiting loop")
        print(str(len(listing_info)) + " vendors discovered")
        break

No more pages remaining exiting loop
760 vendors discovered


In [5]:
properties = {"property Name": [], "address":[], "city":[], "state":[], "zip":[], "neighborhood":[], "link":[], "walk":[], "transit":[], "bike":[], "sound":[]}
listing = {"property Name":[], "unit":[], "rent":[], "bedrooms":[], "bathrooms":[], "squarefeet":[], "availibility":[]}

In [6]:
for link in range(len(listing_info)):
    
    propname=""
    try:
        driver.get(listing_info[link])
        propname = driver.find_element_by_class_name("propertyNameRow").text
    except:
        continue   
        
    properties["link"].append(listing_info[link])
    properties["property Name"].append(propname)
    address = driver.find_element_by_class_name("propertyAddressContainer").text.split(", ")
    statezipneighborhood = address[2].split('\n')
    statezip = statezipneighborhood[0].split()
    properties["address"].append(address[0])
    properties["city"].append(address[1])
    properties["state"].append(statezip[0])
    properties["zip"].append(statezip[1])
    properties["neighborhood"].append(statezipneighborhood[1])
    
    ht = [".1",".2",".3",".4",".5",".6",".7",".8",".9","1"]
    for h in ht: 
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight*{})".format(h))
        time.sleep(.25)   

    score = driver.find_elements_by_class_name("score") 
    #maping [0] = walk, [1] = transit, [2] = bike, [3] = sound
    try:
        properties["walk"].append(score[0].text)
    except:
        properties["walk"].append(None)
    try:
        properties["transit"].append(score[1].text)
    except:
        properties["transit"].append(None)
    try:
        properties["bike"].append(score[2].text)
    except:
        properties["bike"].append(None)
    try:
        properties["sound"].append(score[3].text)
    except:
        properties["sound"].append(None)
        
    driver.execute_script("window.scrollTo(0,0)")

    try:
        types = driver.find_element_by_xpath('//*[@id="bedsFilterContainer"]/ul')
        tabs = types.find_elements_by_tag_name("li")
        for tab in tabs[1:]:
            tab.click()
            grids = driver.find_elements_by_class_name("pricingGridItem")
            for grid in grids:
                rentals = grid.find_elements_by_class_name("grid-container")
                for r in rentals:
                    info = r.text.split("\n")
                    if len(info) > 4:
                        listing["bedrooms"].append(tab.get_attribute("data-tab-id")[3:])
                        listing["bathrooms"].append(
                            grid.find_element_by_class_name("detailsTextWrapper").text.split(",")[1].split()[0])
                        listing["unit"].append(info[1])
                        listing["rent"].append(info[3])
                        listing["squarefeet"].append(info[5])
                        listing["availibility"].append(info[7])
                        listing["property Name"].append(propname)
    except:
        try:
            grid = driver.find_element_by_class_name("pricingGridItem")
            info = grid.find_element_by_class_name("detailsLabel").text.split()
            listing["bedrooms"].append(info[0])
            listing["bathrooms"].append(info[2])
            listing["unit"].append(grid.find_element_by_class_name("modelName").text)
            listing["rent"].append(grid.find_element_by_class_name("rentLabel").text)
            listing["squarefeet"].append(info[4])
            listing["availibility"].append(info[-2]+" "+info[-1])
            listing["property Name"].append(propname)
        except:
            for key in properties.keys():
                del properties[key][-1]
            continue

In [8]:
properties_df = pd.DataFrame(data= properties)

listing_df = pd.DataFrame(data= listing)

In [12]:
from datetime import datetime
properties_df.to_csv('Seattle Properties '+ str(datetime.now()).replace(":", " ").replace(".", " ") +".csv")
listing_df.to_csv('Seattle listings '+ str(datetime.now()).replace(":", " ").replace(".", " ") +".csv")

In [10]:
driver.close()